In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import math

from sklearn.model_selection import train_test_split, KFold, GroupKFold
from sklearn.metrics import mean_squared_error, r2_score
from random import randint
from src.utils import split_in, extended_describe

In [ ]:
df_users = pd.read_csv('../data/01_raw/usuarios.csv')
df_books = pd.read_csv('../data/01_raw/libros.csv')
df_train = pd.read_csv('../data/01_raw/opiniones_train.csv')
df_test = pd.read_csv('../data/01_raw/opiniones_test.csv')

Cada dataset completo con la cantidad de opiniones

In [ ]:
opinions_by_user = df_train.groupby('usuario').agg('count').reset_index()[['usuario', 'libro']].rename(columns={'libro': 'cantidad_opiniones'})
opinions_by_user_test = df_test.groupby('usuario').agg('count').reset_index()[['usuario', 'libro']].rename(columns={'libro': 'cantidad_opiniones'})

In [ ]:
df_train = df_train.merge(opinions_by_user, left_on='usuario', right_on='usuario')
df_test = df_test.merge(opinions_by_user_test, left_on='usuario', right_on='usuario')

In [ ]:
df_train.to_csv('../data/02_intermediate/opiniones_train_opiniones_1.csv', index=False)
df_test.to_csv('../data/02_intermediate/opiniones_test_opiniones_1.csv', index=False)

Pongo en test la cantidad de opiniones de train y completo los na con 0

In [ ]:
df_train = pd.read_csv('../data/01_raw/opiniones_train.csv')
df_test = pd.read_csv('../data/01_raw/opiniones_test.csv')

In [ ]:
opinions_by_user = df_train.groupby('usuario').agg('count').reset_index()[['usuario', 'libro']].rename(columns={'libro': 'cantidad_opiniones'})

In [ ]:
df_train = df_train.merge(opinions_by_user, left_on='usuario', right_on='usuario')
df_test = df_test.merge(opinions_by_user, left_on='usuario', right_on='usuario')

In [ ]:
df_test[df_test.cantidad_opiniones.isna()]

In [ ]:
df_test.to_csv('../data/02_intermediate/opiniones_test_opiniones_2.csv', index=False)

In [ ]:
a=pd.DataFrame({'a': [11,12], 'b': [21,22], 'c': [31,32]})
x=pd.Series([41,42])

b=pd.DataFrame({'a': [13,14], 'b': [23,24], 'c': [33,34]})
y=pd.Series([43,44])

In [ ]:
aa=pd.concat([a, pd.DataFrame(x)], axis=1)
bb=pd.concat([b, pd.DataFrame(y)], axis=1)

In [ ]:
aa

In [ ]:
bb

In [ ]:
pd.concat([a,b], axis=0)

In [ ]:
pd.concat([x,y], axis=0)

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
from src.light_gbm import PredictLightGbm
from sklearn.model_selection import train_test_split
df_train = pd.read_csv("../data/02_intermediate/opiniones_train_opiniones_1.csv").head(200)
df_test = pd.read_csv("../data/02_intermediate/opiniones_test_opiniones_2.csv").head(200)
df_train["libro"] = df_train["libro"].astype("category")
df_test["libro"] = df_test["libro"].astype("category")

In [2]:
X_train, X_test, y_train, y_test = train_test_split(df_train.drop(columns=["puntuacion"], axis=1),df_train.puntuacion,test_size=0.2,random_state=0,)

In [3]:
predictor = PredictLightGbm()
predictor.tune(X_train, y_train)
predictor.train(X_train, y_train)
y_prediction = predictor.predict(X_test)
print(y_prediction)


2020-11-28 19:12:25,775  lightgbm: INFO: Entering tune ...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.6s finished
2020-11-28 19:12:27,490  lightgbm: INFO: Best score tunning: -0.014364523625876258
2020-11-28 19:12:27,490  lightgbm: INFO: Best params tunning: {'subsample': 0.9, 'reg_lambda': 20, 'reg_alpha': 0.1, 'random_state': 42, 'num_leaves': 4, 'n_estimators': 1000, 'min_sum_hessian_in_leaf': 0.0004, 'min_split_gain': 0.01, 'metric': 'rmse', 'max_depth': 20, 'max_bin': 20, 'learning_rate': 0.01, 'colsample_bytree': 0.7, 'boosting_type': 'dart'}
2020-11-28 19:12:27,513  lightgbm: INFO: rmse in validation: 1.434807816185762
2020-11-28 19:12:27,515  lightgbm: INFO: Entering train ...
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.0004, min_child_weight=0.001 will be

In [5]:
#predictor.train(X_train, y_train)
predictor.predict(X_train)

2020-11-28 19:00:43,115  lightgbm: INFO: Entering predict ...


In [ ]:
rmse = predictor.rmse(y_test, y_prediction)
logger.info(f"lightgbm_rmse in test: {rmse}")

all_x = pd.concat([X_train, X_test], axis=0)
all_y = pd.concat([y_train, y_test], axis=0)
predictor.train(all_x, all_y)

In [6]:
import lightgbm

In [17]:
def lgmb():
    gbm = lightgbm.LGBMRegressor()
    gbm.set_params(**predictor.best_params())
    gbm.fit(X_train, y_train)
    return gbm.predict(X_test)
    

In [19]:
lgmb()

[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.001


array([7.06875, 7.06875, 7.06875, 7.06875, 7.06875, 7.06875, 7.06875,
       7.06875, 7.06875, 7.06875, 7.06875, 7.06875, 7.06875, 7.06875,
       7.06875, 7.06875, 7.06875, 7.06875, 7.06875, 7.06875, 7.06875,
       7.06875, 7.06875, 7.06875, 7.06875, 7.06875, 7.06875, 7.06875,
       7.06875, 7.06875, 7.06875, 7.06875, 7.06875, 7.06875, 7.06875,
       7.06875, 7.06875, 7.06875, 7.06875, 7.06875])

In [10]:
X_train.iloc[:,1]

134    216
66     216
26     216
113    216
168    216
      ... 
67     216
192    216
117    216
47     216
172    216
Name: usuario, Length: 160, dtype: int64